In [1]:
# STEP 1: Setup Environment

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install -U torch torchvision torchaudio
!pip install -U opencv-python-headless pycocotools labelme2coco
!pip install git+https://github.com/facebookresearch/detectron2.git


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully 

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# STEP 1–4 Combined: Mount, Convert, Train

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
from labelme2coco import get_coco_from_labelme_folder, save_json
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2 import model_zoo

# Paths
train_dir = "/content/drive/MyDrive/AI_4/Step2Training"
val_dir = "/content/drive/MyDrive/AI_4/Step2Testing"
output_dir = "/content/drive/MyDrive/AI_4/coco_annotations"
os.makedirs(output_dir, exist_ok=True)

# Convert LabelMe → COCO
train_coco = get_coco_from_labelme_folder(train_dir, category_id_start=1)
save_json(train_coco.json, os.path.join(output_dir, "train.json"))

val_coco = get_coco_from_labelme_folder(val_dir, coco_category_list=train_coco.json_categories, category_id_start=1)
save_json(val_coco.json, os.path.join(output_dir, "val.json"))

# Register datasets
register_coco_instances("wood_train", {}, os.path.join(output_dir, "train.json"), train_dir)
register_coco_instances("wood_val", {}, os.path.join(output_dir, "val.json"), val_dir)

# Setup Detectron2 Config
setup_logger()
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TRAIN = ("wood_train",)
cfg.DATASETS.TEST = ("wood_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(train_coco.json_categories)
cfg.OUTPUT_DIR = "/content/drive/MyDrive/AI_4/output_mask_rcnn"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Train
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
There are 590 listed files in folder Step2Training.


Converting labelme annotations to COCO format: 100%|██████████| 590/590 [01:56<00:00,  5.06it/s]


There are 10 listed files in folder Step2Testing.


Converting labelme annotations to COCO format: 100%|██████████| 10/10 [00:06<00:00,  1.58it/s]


[04/10 22:08:25 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 121MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[04/10 22:08:27 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/10 22:08:40 d2.utils.events]:  eta: 0:06:46  iter: 19  total_loss: 2.133  loss_cls: 0.6268  loss_box_reg: 0.7419  loss_mask: 0.699  loss_rpn_cls: 0.03033  loss_rpn_loc: 0.02811    time: 0.4343  last_time: 0.4999  data_time: 0.0286  last_data_time: 0.0052   lr: 4.9953e-06  max_mem: 2047M
[04/10 22:08:58 d2.utils.events]:  eta: 0:06:46  iter: 39  total_loss: 2.132  loss_cls: 0.5999  loss_box_reg: 0.79  loss_mask: 0.6765  loss_rpn_cls: 0.03228  loss_rpn_loc: 0.02794    time: 0.4409  last_time: 0.4220  data_time: 0.0145  last_data_time: 0.0079   lr: 9.9902e-06  max_mem: 2047M
[04/10 22:09:07 d2.utils.events]:  eta: 0:06:34  iter: 59  total_loss: 2.05  loss_cls: 0.5631  loss_box_reg: 0.8173  loss_mask: 0.6325  loss_rpn_cls: 0.02293  loss_rpn_loc: 0.02829    time: 0.4386  last_time: 0.5032  data_time: 0.0099  last_data_time: 0.0074   lr: 1.4985e-05  max_mem: 2047M
[04/10 22:09:15 d2.utils.events]:  eta: 0:06:28  iter: 79  total_loss: 1.928  loss_cls: 0.5062  loss_box_reg: 0.8076  loss_ma

In [3]:
# Inference & visualization imports
import random
import cv2
import os
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Create predictor using trained config
predictor = DefaultPredictor(cfg)

# Define test image directory and output directory
test_images_dir = val_dir  # Step2Testing
output_vis_dir = "/content/drive/MyDrive/AI_4/test_predictions"
os.makedirs(output_vis_dir, exist_ok=True)

# List test image files
image_files = [f for f in os.listdir(test_images_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

# Run inference and save visualized outputs
for image_file in image_files:
    image_path = os.path.join(test_images_dir, image_file)
    image = cv2.imread(image_path)

    outputs = predictor(image)

    visualizer = Visualizer(image[:, :, ::-1],
                            metadata=MetadataCatalog.get("wood_val"),
                            scale=1.0)

    vis_output = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = vis_output.get_image()[:, :, ::-1]

    output_path = os.path.join(output_vis_dir, f"pred_{image_file}")
    cv2.imwrite(output_path, result_image)

print(f"Done. Predicted images saved to: {output_vis_dir}")


[04/10 22:23:32 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


Done. Predicted images saved to: /content/drive/MyDrive/AI_4/test_predictions


In [4]:
import cv2
import os
from detectron2.engine import DefaultPredictor

# Reuse the predictor (or re-create it if needed)
predictor = DefaultPredictor(cfg)

# Path to test images
test_images_dir = val_dir  # Step2Testing

# List image files
image_files = [f for f in os.listdir(test_images_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

# Dictionary to hold image name and detected count
log_counts = {}

# Loop through and count logs
for image_file in image_files:
    image_path = os.path.join(test_images_dir, image_file)
    image = cv2.imread(image_path)

    outputs = predictor(image)
    instances = outputs["instances"]
    num_logs = len(instances)

    log_counts[image_file] = num_logs
    print(f"{image_file}: {num_logs} log(s) detected")

# Optional: save the counts to a text file
output_txt = "/content/drive/MyDrive/AI_4/log_counts.txt"
with open(output_txt, "w") as f:
    for image_name, count in log_counts.items():
        f.write(f"{image_name}: {count} log(s)\n")

print(f"\nLog counts saved to {output_txt}")


[04/10 22:33:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


00300-ZED-Left-1622128958.png: 100 log(s) detected
00300-ZED-Right-1622128958.png: 100 log(s) detected
00301-ZED-Left-1622129085.png: 100 log(s) detected
00301-ZED-Right-1622129085.png: 100 log(s) detected
00302-ZED-Left-1622129159.png: 100 log(s) detected
00302-ZED-Right-1622129159.png: 100 log(s) detected
00303-ZED-Left-1622129219.png: 100 log(s) detected
00303-ZED-Right-1622129219.png: 100 log(s) detected
00304-ZED-Left-1622129266.png: 100 log(s) detected
00304-ZED-Right-1622129266.png: 100 log(s) detected

Log counts saved to /content/drive/MyDrive/AI_4/log_counts.txt
